<a href="https://colab.research.google.com/github/mdluthfi1/awesome/blob/main/Quantitative_Finance_Black_Litterman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from datetime import datetime,timedelta

plt.style.use('seaborn-v0_8')

In [20]:
tickers = ['SPY', 'TLT', 'GLD']
end_date= datetime.today()
start_date = end_date - timedelta(days=365*3)

stock_data = yf.download(tickers = tickers, start = start_date, end = end_date, auto_adjust = True)['Close']
print(stock_data.head())

market_cap = []
for symbol in tickers:
    t = yf.Ticker(symbol)
    marketCap = t.info.get('marketCap')
    print(f"{symbol} Market Cap: {marketCap}")
    market_cap.append(marketCap)
print(market_cap)

[*********************100%***********************]  3 of 3 completed


Ticker             GLD         SPY        TLT
Date                                         
2022-11-07  155.850006  364.637634  83.121735
2022-11-08  159.449997  366.604919  84.030655
2022-11-09  158.649994  359.052155  84.306885
2022-11-10  163.479996  378.783539  87.550484
2022-11-11  164.559998  382.449585  87.229691
SPY Market Cap: 627157172224
TLT Market Cap: 9844477952
GLD Market Cap: 95993430016
[627157172224, 9844477952, 95993430016]


In [5]:
returns = np.log(stock_data/(stock_data.shift(1))).dropna()
print(returns.describe)

<bound method NDFrame.describe of Ticker           GLD       SPY       TLT
Date                                    
2022-11-08  0.022836  0.005381  0.010875
2022-11-09 -0.005030 -0.020817  0.003282
2022-11-10  0.029990  0.053497  0.037752
2022-11-11  0.006585  0.009632 -0.003671
2022-11-14  0.002185 -0.008543 -0.002455
...              ...       ...       ...
2025-10-28 -0.007192  0.002652  0.002612
2025-10-29 -0.003794  0.000480 -0.010158
2025-10-30  0.019451 -0.011059 -0.005835
2025-10-31 -0.005445  0.003275 -0.002986
2025-11-03  0.001791  0.001875 -0.002504

[749 rows x 3 columns]>


In [6]:
annual_return = returns.mean() * 252
cov_matrix = returns.cov() * 252
annual_std = returns.std() * 252

print(f"Annualised Expected Return [E(R)] = {annual_return}")
print(f"Annualised Covariance Matrix = {cov_matrix}")
print(f"Annualised Standard Deviation = {returns.std}")

Annualised Expected Return [E(R)] = Ticker
GLD    0.289785
SPY    0.211320
TLT    0.025775
dtype: float64
Annualised Covariance Matrix = Ticker       GLD       SPY       TLT
Ticker                              
GLD     0.026208  0.002603  0.005936
SPY     0.002603  0.025296  0.003104
TLT     0.005936  0.003104  0.025093
Annualised Standard Deviation = <bound method DataFrame.std of Ticker           GLD       SPY       TLT
Date                                    
2022-11-08  0.022836  0.005381  0.010875
2022-11-09 -0.005030 -0.020817  0.003282
2022-11-10  0.029990  0.053497  0.037752
2022-11-11  0.006585  0.009632 -0.003671
2022-11-14  0.002185 -0.008543 -0.002455
...              ...       ...       ...
2025-10-28 -0.007192  0.002652  0.002612
2025-10-29 -0.003794  0.000480 -0.010158
2025-10-30  0.019451 -0.011059 -0.005835
2025-10-31 -0.005445  0.003275 -0.002986
2025-11-03  0.001791  0.001875 -0.002504

[749 rows x 3 columns]>


Calculate implied returns assuming :
  - Market weights: SPY 60%, TLT 30%, GLD 10%
  - Risk aversion δ = 3.0

In [7]:
market_weights = np.array([0.6,0.3,0.1])
delta = 3.0

def implied_return(delta,market_weights,cov_matrix):
    imp_return = delta* cov_matrix @ market_weights
    return imp_return

Sigma = implied_return(delta,market_weights,cov_matrix)
print(Sigma)

Ticker
GLD    0.051298
SPY    0.028382
TLT    0.021006
dtype: float64


Intrepretation: The market implies SPY will return 5.19% above risk-free rate

Express this view: "SPY will outperform TLT by 4%, with 80% confidence"

In [8]:
rho_spy_tlt = cov_matrix.loc['SPY','TLT']
print(rho_spy_tlt)

spy_volatilty = annual_std.loc['SPY']
tlt_volatility = annual_std.loc['TLT']

volatility_spy_tlt = rho_spy_tlt * spy_volatilty * tlt_volatility
print(f"The /sigma_spytlt = {volatility_spy_tlt}")

0.00310407947071852
The /sigma_spytlt = 0.01970752028974468


In [36]:
def calculate_equilibrium_returns(risk_aversion, cov_matrix, market_cap) -> np.ndarray:
    market_weights = market_cap / np.sum(market_cap)
    return risk_aversion * cov_matrix @ market_weights

def add_relative_view(outperformer, underperformer, excess_return, confidence, tau, cov_matrix, tickers, equilibrium_returns):
    n_assets = len(tickers)
    P = np.zeros((1, n_assets))
    outperformer_index = tickers.index(outperformer)
    underperformer_index = tickers.index(underperformer)
    P[0, outperformer_index] = 1
    P[0, underperformer_index] = -1

    Q = np.array([excess_return])
    z_score = abs(np.percentile(np.random.randn(10000), confidence * 100))
    view_variance = (P @ cov_matrix @ P.T) / (z_score ** 2)
    Omega = view_variance * tau

    return combine_views(tau, cov_matrix, P, Q, Omega, equilibrium_returns)

def combine_views(tau, cov_matrix, P, Q, Omega, equilibrium_returns):
    term1 = tau * cov_matrix @ P.T
    term2 = np.linalg.inv(P @ tau * cov_matrix @ P.T + Omega)
    term3 = Q - P @ equilibrium_returns
    return equilibrium_returns + term1 @ term2 @ term3



In [39]:
tau = 0.025
risk_aversion = 2.5
assets = tickers
n_assets = len(assets)

equilibrium_returns = calculate_equilibrium_returns(
    risk_aversion=risk_aversion,
    cov_matrix=cov_matrix,
    market_cap=market_cap)


posterior = add_relative_view(
    'SPY', 'TLT', 0.04, 0.8, tau,
    cov_matrix=cov_matrix,
    tickers=tickers,
    equilibrium_returns=equilibrium_returns
)


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)